# 用CNN实现MNIST数字识别

In [1]:
import os 
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Activation
import numpy as np

In [2]:
(X_train,y_train),(X_test,y_test)=mnist.load_data()
X_train.shape

(60000, 28, 28)

In [3]:
X_train4D=X_train.reshape(X_train.shape[0],28,28,1).astype('float32')
X_test4D = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')

#归一化
X_train4D_Normalize = X_train4D / 255 # 归一化
X_test4D_Normalize = X_test4D / 255

#将类别向量映射为二值类别矩阵,将原有的类别向量转换为独热编码的形式
y_trainOnehot=to_categorical(y_train)
y_testOnehot = to_categorical(y_test)

In [4]:
#建立模型
model = Sequential()

#一层卷积
model.add(Conv2D(filters=16,kernel_size=(5,5),padding='SAME',input_shape=(28,28,1),activation='relu'))

#池化+Dropout
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#二层卷积
model.add(Conv2D(filters=32,kernel_size=(5,5),padding='SAME',activation='relu'))

#池化+Dropout
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#三层卷积
model.add(Conv2D(filters=64,kernel_size=(5,5),padding='SAME',activation='relu'))

#四层卷积
model.add(Conv2D(filters=128,kernel_size=(5,5),padding='SAME',activation='relu'))

#池化+Dropout
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#全连接层
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        12832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          5

In [5]:
#编译模型
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

#训练模型
'''
X_train4D_Normalize：数字图像的特征值
y_trainOnehot：真是标签
validation_split：训练与验证数据比例：80%用作训练数据，20%用作验证数据
epochs：训练周期
batch_size：每批次的数据项数
verbose=2：显示训练过程
'''
train_history=model.fit(x=X_train4D_Normalize,y=y_trainOnehot,validation_split=0.2,batch_size=300,epochs=10,verbose=2)
score=model.evaluate(X_test4D_Normalize,y_testOnehot)[1]

Epoch 1/10
160/160 - 80s - loss: 0.4714 - accuracy: 0.8443 - val_loss: 0.0984 - val_accuracy: 0.9703
Epoch 2/10
160/160 - 68s - loss: 0.1210 - accuracy: 0.9622 - val_loss: 0.0607 - val_accuracy: 0.9818
Epoch 3/10
160/160 - 31s - loss: 0.0829 - accuracy: 0.9751 - val_loss: 0.0508 - val_accuracy: 0.9847
Epoch 4/10
160/160 - 27s - loss: 0.0641 - accuracy: 0.9800 - val_loss: 0.0400 - val_accuracy: 0.9883
Epoch 5/10
160/160 - 66s - loss: 0.0542 - accuracy: 0.9834 - val_loss: 0.0414 - val_accuracy: 0.9874
Epoch 6/10
160/160 - 97s - loss: 0.0471 - accuracy: 0.9853 - val_loss: 0.0335 - val_accuracy: 0.9901
Epoch 7/10
160/160 - 84s - loss: 0.0410 - accuracy: 0.9873 - val_loss: 0.0318 - val_accuracy: 0.9901
Epoch 8/10
160/160 - 30s - loss: 0.0370 - accuracy: 0.9881 - val_loss: 0.0278 - val_accuracy: 0.9910
Epoch 9/10
160/160 - 26s - loss: 0.0322 - accuracy: 0.9895 - val_loss: 0.0305 - val_accuracy: 0.9916
Epoch 10/10
160/160 - 23s - loss: 0.0291 - accuracy: 0.9909 - val_loss: 0.0294 - val_accura

In [6]:
#评估模型
score=model.evaluate(X_test4D_Normalize,y_testOnehot)
score

 14/313 [>.............................] - ETA: 10s - loss: 0.0200 - accuracy: 0.9911

313/313 [==============================] - 11s 35ms/step - loss: 0.0219 - accuracy: 0.9929


[0.02191629633307457, 0.992900013923645]

In [9]:
#预测
prediction = np.argmax(model.predict(X_test4D_Normalize), axis=1)
prediction

array([7, 2, 1, ..., 4, 5, 6], dtype=int64)

In [10]:
pred=model.predict(X_test4D_Normalize)
pred_classes=np.argmax(pred,axis=1)
pred_classes

array([7, 2, 1, ..., 4, 5, 6], dtype=int64)

In [11]:
pred

array([[7.60870273e-12, 2.26302781e-08, 5.56119240e-08, ...,
        9.99998212e-01, 1.93510452e-09, 1.72921807e-06],
       [4.94327878e-07, 5.81749737e-06, 9.99992728e-01, ...,
        5.08732683e-07, 2.74570823e-07, 2.58923799e-10],
       [1.24316372e-07, 9.99997377e-01, 5.52598820e-08, ...,
        8.21778599e-07, 7.31851571e-07, 2.34171367e-07],
       ...,
       [1.22659023e-12, 1.83145978e-08, 1.40558884e-10, ...,
        8.69761791e-08, 7.29380289e-09, 2.88953913e-07],
       [7.52222312e-11, 4.70320961e-12, 5.36388338e-12, ...,
        1.52011251e-11, 1.27868352e-05, 9.38484774e-08],
       [6.90285304e-08, 2.17292937e-11, 4.11846912e-09, ...,
        1.28663495e-11, 9.49841805e-09, 1.29880121e-10]], dtype=float32)

In [10]:
from keras.preprocessing.image import ImageDataGenerator

# 数据增强
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(X_train4D_Normalize)

# 构建和训练改进的CNN模型
#建立模型
model = Sequential()

#一层卷积
model.add(Conv2D(filters=16,kernel_size=(5,5),padding='SAME',input_shape=(28,28,1),activation='relu'))

#池化+Dropout
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#二层卷积
model.add(Conv2D(filters=32,kernel_size=(5,5),padding='SAME',activation='relu'))

#池化+Dropout
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#三层卷积
model.add(Conv2D(filters=64,kernel_size=(5,5),padding='SAME',activation='relu'))

#四层卷积
model.add(Conv2D(filters=128,kernel_size=(5,5),padding='SAME',activation='relu'))

#池化+Dropout
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#全连接层
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10,activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
train_history_improved = model.fit(datagen.flow(X_train4D_Normalize, y_trainOnehot, batch_size=300), validation_data=(X_test4D_Normalize, y_testOnehot), epochs=20, verbose=2)
score_improved = model.evaluate(X_test4D_Normalize, y_testOnehot)[1]
score_improved

Epoch 1/20
200/200 - 44s - loss: 0.6005 - accuracy: 0.7981 - val_loss: 0.0748 - val_accuracy: 0.9751
Epoch 2/20
200/200 - 61s - loss: 0.1574 - accuracy: 0.9529 - val_loss: 0.0391 - val_accuracy: 0.9873
Epoch 3/20
200/200 - 59s - loss: 0.1044 - accuracy: 0.9675 - val_loss: 0.0289 - val_accuracy: 0.9900
Epoch 4/20
200/200 - 61s - loss: 0.0826 - accuracy: 0.9746 - val_loss: 0.0272 - val_accuracy: 0.9906
Epoch 5/20
200/200 - 58s - loss: 0.0722 - accuracy: 0.9780 - val_loss: 0.0207 - val_accuracy: 0.9923
Epoch 6/20
200/200 - 60s - loss: 0.0666 - accuracy: 0.9800 - val_loss: 0.0231 - val_accuracy: 0.9920
Epoch 7/20
200/200 - 61s - loss: 0.0594 - accuracy: 0.9825 - val_loss: 0.0239 - val_accuracy: 0.9905
Epoch 8/20
200/200 - 60s - loss: 0.0551 - accuracy: 0.9834 - val_loss: 0.0187 - val_accuracy: 0.9932
Epoch 9/20
200/200 - 57s - loss: 0.0532 - accuracy: 0.9844 - val_loss: 0.0169 - val_accuracy: 0.9939
Epoch 10/20
200/200 - 61s - loss: 0.0460 - accuracy: 0.9861 - val_loss: 0.0172 - val_accura

0.9952999949455261

In [12]:
score=model.evaluate(X_test4D_Normalize,y_testOnehot)
score

313/313 [==============================] - 2s 7ms/step - loss: 0.0149 - accuracy: 0.9953


[0.01488664373755455, 0.9952999949455261]